In [ ]:
%pip install -r requirements.txt

### Запуск пайплайна

In [6]:
from process.pipeline import ParaphrasePipeline
from process import modules

In [ ]:
p = ParaphrasePipeline(modules=[
    modules.CapitalizeSubjectsModule(),
])
p.load()

In [8]:
p.run(["Это первое предложение.", "Это второе предложение."])

['ЭТО первое предложение.', 'ЭТО второе предложение.']

### Добавление нового модуля

Один модуль - один скрипт (пример - `process/modules/capitalize_subjects.py`).

1. Положить скрипт в `process/modules/`.
2. Добавить импорт нового класса в файл `process/modules/__init__.py`

In [9]:
from typing import List

from process.module import ParaphraseModule
from process.preprocessing_utils import PreprocessingUtils


class CapitalizeVerbsModule(ParaphraseModule):
    def __init__(self, name="capitalize_nouns") -> None:
        super().__init__(name=name)
    
    def load(self, preproc_utils: PreprocessingUtils) -> None:
        # load any tools as `preproc_utils` attributes
        self.loaded = True
    
    def process(self, input_text: str, preproc_utils: PreprocessingUtils) -> str:
        output_text = ""
        last_char = 0
        parsed_sent = preproc_utils.stanza_parse(input_text)
        for word in parsed_sent.words:
            if word.upos == "VERB":
                word.text = word.text.upper()
            if word.start_char > last_char:
                output_text += " "
            output_text += word.text
            last_char = word.end_char
        return output_text

    def process_batch(self, inputs: List[str], preproc_utils: PreprocessingUtils) -> List[str]:
        outputs = []
        for input_text in inputs:
            outputs.append(self.process(input_text, preproc_utils))
        return outputs

# эти строки нужно добавить в конце скрипта модуля
# if __name__ == "__main__":
#     print("This module is not callable")
    # exit()

In [ ]:
p = ParaphrasePipeline(modules=[
    CapitalizeVerbsModule(),  # пока класс создан только локально, обращаемся к нему без префикса `modules.`, а при импорте из библиотеки уже нужно будет добавить префикс
])
p.load()

In [11]:
p.run(["Если я чешу в затылке, не беда!"])

['Если я ЧЕШУ в затылке, не беда!']